In [1]:
eta = 5
optimizer_lr = 1e-5 # try 5e-6

total_trial_num = 10        # num trial
n_max_mirror_iterations=25       # num mirror steps
n_max_iterations=1000        # max inner steps
patient_max = 1000          # max patient

stopping_norm = 1e-4
num_samples = 1000         # number of theta generated

verbose = True
epcoh_min = 200
set_seed = 114540 #
pretrain_factor = 9

outer_lr_final_factor = 1e-1
outer_eta_final_factor = 1e0
import math
outer_lr_factor = pow(math.e, math.log(outer_lr_final_factor)/n_max_mirror_iterations)
outer_eta_factor = pow(math.e, math.log(outer_eta_final_factor)/n_max_mirror_iterations)

In [2]:
!pip install normflows

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.8/64.8 kB 1.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for normflows: filename=normflows-1.7.2-py2.py3-none-any.whl size=86917 sha256=99eaee971b506a7204f394756df316df5464a36a323564d87727708dff4ab5a3
  Stored in directory: /root/.cache/pip/wheels/8a/a4/89/3e09f53a561355c45eccfebeffc07a0e34d36a3f41e3ef68a3
Successfully built normflows


In [3]:
# Import required packages
import torch
import numpy as np
import normflows as nf
import os
import numpy as np
from matplotlib import pyplot as plt
from tqdm import tqdm
from IPython.display import clear_output
import torch.nn.functional as F
import torch.distributions as TD
import pandas as pd
import seaborn as sb
import torch.nn as nn
import shutil
import gc
import copy
import time
from scipy.optimize import linear_sum_assignment

In [4]:
torch.manual_seed(114514)
# Move model on GPU if available
enable_cuda = True
device = torch.device('cuda' if torch.cuda.is_available() and enable_cuda else 'cpu')

In [5]:
class Target(nn.Module):
    """
    Sample target distributions to test models
    """

    def __init__(self, prop_scale=torch.tensor(6.0), prop_shift=torch.tensor(-3.0)):
        """Constructor

        Args:
          prop_scale: Scale for the uniform proposal
          prop_shift: Shift for the uniform proposal
        """
        super().__init__()
        self.register_buffer("prop_scale", prop_scale)
        self.register_buffer("prop_shift", prop_shift)

    def log_prob(self, z):
        """
        Args:
          z: value or batch of latent variable

        Returns:
          log probability of the distribution for z
        """
        raise NotImplementedError("The log probability is not implemented yet.")

    def rejection_sampling(self, num_steps=1):
        """Perform rejection sampling on image distribution

        Args:
          num_steps: Number of rejection sampling steps to perform

        Returns:
          Accepted samples
        """
        eps = torch.rand(
            (num_steps, self.n_dims),
            dtype=self.prop_scale.dtype,
            device=self.prop_scale.device,
        )
        z_ = self.prop_scale * eps + self.prop_shift
        prob = torch.rand(
            num_steps, dtype=self.prop_scale.dtype, device=self.prop_scale.device
        )
        prob_ = torch.exp(self.log_prob(z_) - self.max_log_prob)
        accept = prob_ > prob
        z = z_[accept, :]
        return z

    def sample(self, num_samples=1):
        """Sample from image distribution through rejection sampling

        Args:
          num_samples: Number of samples to draw

        Returns:
          Samples
        """
        z = torch.zeros(
            (0, self.n_dims), dtype=self.prop_scale.dtype, device=self.prop_scale.device
        )
        while len(z) < num_samples:
            z_ = self.rejection_sampling(num_samples)
            ind = np.min([len(z_), num_samples - len(z)])
            z = torch.cat([z, z_[:ind, :]], 0)
        return z

class TargetDist(Target):
    """
    Bimodal two-dimensional distribution
    """
    def __init__(self):
        super().__init__()
        self.n_dims = 2
        self.max_log_prob = 0.0

    def log_prob(self, z):
        """
        ```
        log(p) = - 1/2 * ((norm(z) - 2) / 0.2) ** 2
                 + log(  exp(-1/2 * ((z[0] - 2) / 0.3) ** 2)
                       + exp(-1/2 * ((z[0] + 2) / 0.3) ** 2))
        ```

        Args:
          z: value or batch of latent variable

        Returns:
          log probability of the distribution for z
        """
        log_prob = (
            -1/4 * (torch.linalg.vector_norm(z, ord=2, dim=1)) ** 4
            - np.log(math.pi ** (3/2))
        )
        return log_prob


In [ ]:
Ln_rho_k_list = np.zeros((total_trial_num, n_max_mirror_iterations))
times_2d = np.zeros((total_trial_num, n_max_mirror_iterations))
W1_2d = np.zeros((total_trial_num, n_max_mirror_iterations))

dim1_normal = TD.MultivariateNormal(
    torch.zeros(1).to(device), 1 * torch.eye(1).to(device))

p = TargetDist()
for trail_num in range(total_trial_num):

  torch.manual_seed(set_seed + trail_num)

  target_sample = p.sample(num_samples)

  mirror_loss_hist = np.array([])
  times = np.array([])
  x_all_data = torch.zeros((n_max_mirror_iterations, num_samples, 2))

  target_pretrain = TD.MultivariateNormal(
      torch.zeros(2).to(device), pretrain_factor * torch.eye(2).to(device))
  # Define 2D Gaussian base distribution
  base = nf.distributions.DiagGaussian(2, trainable=False)
  # Define list of flows
  num_layers = 20
  flows = []
  for i in range(num_layers):
      # Neural network with two hidden layers having 64 units each
      # Last layer is initialized by zeros making training more stable
      param_map = nf.nets.MLP([1, 64, 64, 2], init_zeros=True )
      # Add flow layer
      flows.append(nf.flows.AffineCouplingBlock(param_map))
      # Swap dimensions
      flows.append(nf.flows.Permute(2, mode='swap'))


  # Construct flow model
  model = nf.NormalizingFlow(base, flows).to(device)
  optimizer1 = torch.optim.Adam(model.parameters(), lr=1e-3, weight_decay=1e-5)

  for it in tqdm(range(500)):
      optimizer1.zero_grad()

      # Get training samples
      x = target_pretrain.sample((num_samples,)).to(device)

      # Compute loss
      loss = model.forward_kld(x)

      # Do backprop and optimizer step
      if ~(torch.isnan(loss) | torch.isinf(loss)):
          loss.backward()
          optimizer1.step()

      if it % 100 == 99:
        clear_output(wait=True)
        print('Loss:', loss.item())

  for mirror_itr in range(n_max_mirror_iterations):
    flows = []
    for i in range(num_layers):
        param_map = nf.nets.MLP([1, 64, 64, 2], init_zeros=True )
        flows.append(nf.flows.AffineCouplingBlock(param_map))
        flows.append(nf.flows.Permute(2, mode='swap'))

    model2 = nf.NormalizingFlow(base, flows).to(device)
    model2.load_state_dict(model.state_dict())
    optimizer_lr_input = optimizer_lr * (outer_lr_factor**mirror_itr)
    optimizer = torch.optim.Adam(model2.parameters(), lr=optimizer_lr_input, weight_decay=1e-5)
    current_patient = 0
    epcoh_min = 200

    model2.train()
    torch.cuda.synchronize()
    start_epoch = time.time()

    input_eta = (eta*(outer_eta_factor**mirror_itr))
    for it in tqdm(range(n_max_iterations), disable = not verbose):
      optimizer.zero_grad()
      torch.manual_seed(set_seed + trail_num)
      z = base.sample(num_samples)
      log_prob_rho_0 = base.log_prob(z)
      sampled_theta, log_det_model = model2.forward_and_log_det(z)
      log_prob_model = log_prob_rho_0 - log_det_model
      log_prob_prev = model.log_prob(sampled_theta)

      L_n_loss = torch.mean( (1/4)*torch.linalg.vector_norm(sampled_theta, ord=2, dim=1).to(device) ** 4 + log_prob_model)
      kld_loss = torch.mean(log_prob_model).to(torch.double) - torch.mean(log_prob_prev).to(torch.double)
      kld_loss = kld_loss if kld_loss.item() >= 0 else torch.tensor([0.0]).to(device)
      kld_loss = kld_loss if kld_loss.item() <= 5 else torch.tensor([5.0]).to(device)
      loss = L_n_loss   + (1/input_eta)*kld_loss

      # Do backprop and optimizer step
      if ~(torch.isnan(loss) | torch.isinf(loss)):
        loss.backward()
        grads = [param.grad.detach().flatten()
            for param in model2.parameters()
            if param.grad is not None]
        norm = torch.cat(grads).norm()

        optimizer.step()
        optimizer.zero_grad()


      if verbose:
        if it % 100 == 99:
          # clear_output(wait=True)
          print('trail:',trail_num,'m step:',mirror_itr,'Loss:', loss.item(), ' L_n:', L_n_loss.item(), ' kld:',kld_loss.item(),'norm:', norm.item())

      if norm.item() > epcoh_min:
        current_patient = current_patient + 1

      if norm.item() < epcoh_min:
        epcoh_min = norm.item()
        current_patient = 0

      if (current_patient >= patient_max) or (norm.item() < stopping_norm):
        break

    # After finishing the inner loop
    torch.cuda.synchronize()
    end_epoch = time.time()
    elapsed = end_epoch - start_epoch
    times = np.append(times, elapsed)

    model.load_state_dict(model2.state_dict())


    torch.manual_seed(set_seed + trail_num)
    with torch.no_grad():
      z = base.sample(num_samples)
      log_prob_rho_0 = base.log_prob(z)
      generated1, log_porb = model.forward_and_log_det(z)
    x_all_data[mirror_itr,:,:] = generated1

    L_n_loss_temp = L_n_loss
    mirror_loss_hist = np.append(mirror_loss_hist, L_n_loss_temp.to('cpu').data.numpy())

  # compute W1 loss
  objective_fun_loss_list = np.array([])
  for t in tqdm(range(n_max_mirror_iterations)):
    x = x_all_data[t,:,:]
    x_final = target_sample
    x_data_rep = x.repeat(1,1,num_samples).reshape(num_samples,num_samples,-1)
    cost = torch.norm(x_data_rep - x_final, p = 1, dim = 2).cpu().numpy()
    row_ind, col_ind = linear_sum_assignment(cost)
    temp_loss = (cost[row_ind, col_ind].sum() / num_samples )
    objective_fun_loss_list = np.append(objective_fun_loss_list, temp_loss )

  W1_2d[trail_num,:] = objective_fun_loss_list
  Ln_rho_k_list[trail_num,:] = mirror_loss_hist
  times_2d[trail_num:] = times